# Tensorflow CNN implementation based on the example code of the documentation
### using opencv and a custom saved model for classification

Note: Use Tensorflow 2.4.1  (pip3 install tensorflow==2.4.1)

### How to use:
- download your model weights folder from your google drive (if you trained using google colab) or make sure your model weights fodler are in the right directory for the "model_name"
- "model_name" has to match your weights folder
-  run all cells. 
- if the device (webcam) can't be opened, try changing the devide_number to either -1, 0, 1 or 2. (This depends if your system has an internal camera or not)
- devide_number can also be used to switch between webcams if more than one is available 

Usefull links: <br>
CNN Model is based on: https://www.tensorflow.org/tutorials/images/classification


## Import needed libraries

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

2021-11-24 06:06:58.831792: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib
2021-11-24 06:06:58.831812: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Setup

In [2]:
img_height = 416
img_width = 416
dim = (img_width, img_height)
model_name = "model_1_750E32B"
devide_number = -1
class_names = ['Hammer', 'Workspace']
rotate_frame = True
#Load Model
model = tf.keras.models.load_model(model_name)

2021-11-24 06:06:59.873174: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-24 06:06:59.873817: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-24 06:06:59.899636: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-24 06:06:59.899883: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-11-24 06:06:59.899956: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No su

# Webcam Detection

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import numpy as np
import cv2



cap = cv2.VideoCapture(devide_number)
if not (cap.isOpened()):
    print("Could not open video device")

while(True):    
    # cap frame by frame   
    ret, frame = cap.read()    
    
    # rotate image (if needed), crop image to 416x416 (training data size)
    if (rotate_frame):
        frame = cv2.rotate(frame, cv2.ROTATE_180)
    y = 100
    x = 50
    frame = frame[y:y+img_height, x:x+img_width]


    # convert to tensorflow mat https://stackoverflow.com/questions/40273109/convert-python-opencv-mat-image-to-tensorflow-image-data/40273815
    image = cv2.resize(frame,dsize=(img_height,img_width), interpolation = cv2.INTER_CUBIC)
    np_image_data = np.asarray(image)
    image = np.expand_dims(image, axis=0)

    predictions = model.predict(image)
    score = tf.nn.softmax(predictions[0])

    #print("{} {:.2f}".format(class_names[np.argmax(score)], 100 * np.max(score)))

    if class_names[np.argmax(score)] == "Hammer":
        cv2.putText(frame, "Hammer in Workspace ({:.2f})".format(100 * np.max(score)), (10, 40), 1, 1, (0,255,0), 2) 
    
    else:
       cv2.putText(frame, "No Hammer in Workspace ({:.2f})".format(100 * np.max(score)), (10, 40), 1, 1, (0,0,255), 2)

    cv2.imshow('Press q to exit', frame)
    #Waits for a user input to quit the application  
    if cv2.waitKey(1) & 0xFF == ord('q'):    
        break

cap.release()
cv2.destroyAllWindows()

2021-11-24 06:07:03.920405: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-24 06:07:03.942242: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3999980000 Hz
